In [26]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import pandas as pd
import numpy as np

import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [43]:
# Lee la base de datos
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
    dtypes={"EDAD_CANT": pl.Float64}
)

# Rellena los valores de TIPO EDAD con 99.0 para poder procesarlos
df_nacional = df_nacional.with_columns(
    [pl.col("TIPO_EDAD").fill_null("99.0"), pl.col("EDAD_CANT").fill_null(10000000000000000)]
)

In [44]:
# NeoNato seria
# Con tipo de edad de meses, y entre 0 y 1 mes, o que este expresado en dias u horas
df_neonato = df_nacional.filter(
    ((pl.col("TIPO_EDAD") == "2.0") & (pl.col("EDAD_CANT") <= 1))
    | ((pl.col("TIPO_EDAD") == "3.0") | (pl.col("TIPO_EDAD") == "4.0"))
)

# El otro seria de 1 a 15 anios
df_ninos = df_nacional.filter(
    (pl.col("TIPO_EDAD") == "1.0") & ((pl.col("EDAD_CANT") >= 1) & (pl.col("EDAD_CANT") <= 15))
)

# El otro seria de 15 a 18 anios
df_adolescente = df_nacional.filter(
    (pl.col("TIPO_EDAD") == "1.0") & ((pl.col("EDAD_CANT") > 15) & (pl.col("EDAD_CANT") <= 18))
)

# El completo seria de 0 a 18 anios con tipo de edad 1 Y cualquier egreso con meses, dias u horas
df_total = df_nacional.filter(
    ((pl.col("TIPO_EDAD") == "1.0") & (pl.col("EDAD_CANT") <= 18))
    | (
        (pl.col("TIPO_EDAD") == "2.0")
        | (pl.col("TIPO_EDAD") == "3.0")
        | (pl.col("TIPO_EDAD") == "4.0")
    )
)

In [46]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "PERTENENCIA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

# Define los grupos que se quieren exportar
rankings_a_exportar = {
    "neonato": df_neonato,
    "ninos": df_ninos,
    "adolescente": df_adolescente,
    "hasta_18": df_total,
}

In [47]:
# Itera en los rangos etarios para exportar los rankings
for rango_etario, df_etario in rankings_a_exportar.items():
    print(rango_etario)
    metricas = build_features.obtener_metricas_egresos(df_etario, AGRUPACION).collect()
    metricas.write_csv(f"../data/processed/resumen_egresos_nacionales{rango_etario}.csv")

neonato
ninos
adolescente
hasta_18
